# Python tests

First we need to import the required libraries.

In [18]:
import pandas as pd
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

After that, we can initalize the Azure OpenAI client. This will allow us to connect to the Azure OpenAI model and get completions.

We need to set the following environment variables:

- AZURE_OPENAI_API_KEY
- AZURE_OPENAI_ENDPOINT
- AZURE_OPENAI_MODEL_VERSION

For this we will use dotenv to load the environment variables.

In [19]:
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

Next we can create a chat completion function to use on our model.

In [20]:
# Example function to connect to Azure OpenAI model
def get_completion(prompt, model="gpt-4.1"):
    """
    Get completion from Azure OpenAI model
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000,
        temperature=0.7
    )
    return response.choices[0].message.content

Now we will test the connection to the Azure OpenAI model.

Edit your prompt to test the connection to the Azure OpenAI model.

```html
<input type="text" name="name" id="name" value="John Doe">
```

In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
query_input = widgets.Text(
    value='Give me icd 10 codes for diabetes.',
    placeholder='Type your query',
    description='Question:',
    layout=widgets.Layout(width="80%", height="75px",padding="10px")
)

# Display the widget in the cell
display(query_input)

Text(value='Give me icd 10 codes for diabetes.', description='Question:', layout=Layout(height='75px', padding…

In [40]:
def build_prompt(question: str) -> str:
    return f"""
You are a careful analyst. Follow the instructions precisely.

# Input
Question:
{question}

# Goal
- Answer the question by producing a table that contains **exactly** the fields requested in the question (and only those fields, in that order).
- After the table, provide a short prose section titled **"Exclusions & Rationale"** that explains which records were excluded and why.

# Output Format (strict)
1) First, output a Markdown table with a header row. The columns must match the requested fields **exactly** (names and order).
2) Then output the section:
   ### Exclusions & Rationale
   - Briefly describe any records considered but excluded, with concrete reasons (e.g., missing required field, outside date range, duplicate, low confidence, conflicting source, not matching filters).
   - If nothing was excluded, state: "No records excluded."

# Rules
- Do **not** add extra columns or derived fields unless explicitly requested. If the question is ambiguous about fields, infer the minimal set and clearly state the assumption in **Exclusions & Rationale**.
- Do **not** fabricate or guess values. If a specific field value is unavailable, leave the cell blank and explain the gap in **Exclusions & Rationale**.
- Ensure consistent units, formats, and identifiers across the table (dates, currencies, IDs).
- If no records match, output a header-only empty table (with the requested columns) and explain why no records qualified.
- Remove duplicates; if deduping occurred, explain your approach in **Exclusions & Rationale**.
- If the question implies filters (time ranges, categories, thresholds), apply them and mention them in **Exclusions & Rationale**.

# Reflective Check (do this before finalizing)
- Column names exactly match the requested fields and are in the correct order.
- Every row adheres to all implied/explicit filters from the question.
- No invented values; blanks are used where data is missing and reasons are documented.
- No duplicates; totals and counts (if present) are internally consistent.
- Units and formats are uniform (e.g., ISO-8601 dates, consistent currency symbols).
- The **Exclusions & Rationale** section clearly lists exclusion reasons or states none.

# Final Deliverable
Return only:
1) The Markdown table.
2) The "### Exclusions & Rationale" section.
No additional commentary.
"""

my_prompt: str = build_prompt(query_input.value)

response = get_completion(my_prompt)
print(response)

| ICD-10 Code | SNOMED Code | Description                                           |
|-------------|-------------|------------------------------------------------------|
| E10         | 46635009    | Type 1 diabetes mellitus                             |
| E11         | 44054006    | Type 2 diabetes mellitus                             |
| E13         | 58718002    | Other specified diabetes mellitus                    |
| E14         | 73211009    | Unspecified diabetes mellitus                        |

### Exclusions & Rationale
- Gestational diabetes (ICD-10: O24) and neonatal diabetes (ICD-10: P70.2, P70.3) were excluded as requested.
- Subcategories (e.g., E10.1, E11.2) were not listed individually for brevity, as the question requested general codes and descriptions.
- No records excluded due to missing values or duplication.
- No invented values; all SNOMED codes and descriptions verified.


### These results are run against gpt-4.1 without any system prompt or RAG data. Add those to this could yield better results, while requiring more work on the naitenance side.

### Now we need to try a more complex task. A chat interface.

In [5]:
# Chat UI in Jupyter powered by Azure OpenAI + .env
# !pip install openai python-dotenv ipywidgets

import os, html, textwrap
from dotenv import load_dotenv
from ipywidgets import (
    VBox, HBox, Textarea, Button, HTML, Layout, RadioButtons, Label
)
from IPython.display import display
from openai import AzureOpenAI

# --- Load environment variables (expects a .env file next to your notebook) ---
load_dotenv()

AZURE_OPENAI_API_KEY   = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT  = os.getenv("AZURE_OPENAI_ENDPOINT")   # e.g. https://my-res.openai.azure.com/
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_MODEL_VERSION")  # e.g. 2024-10-21
AZURE_OPENAI_DEPLOYMENT  = os.getenv("AZURE_DEPLOYMENT_NAME")   # your deployment name (not the model family)
# Chat UI with sidebar format selector + prompt template (Azure OpenAI)
# !pip install openai python-dotenv ipywidgets

if not all([AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_VERSION, AZURE_OPENAI_DEPLOYMENT]):
    raise RuntimeError("Missing one or more Azure OpenAI ENV VARS.")

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
)

# --- UI: sidebar + main ---
format_picker = RadioButtons(
    options=[("CSV (default)", "csv"), ("JSON", "json"), ("Markdown", "markdown")],
    value="csv"
)
sidebar = VBox(
    [
        HTML("<h3 style='margin:0 0 6px 0;'>Output format</h3>"),
        format_picker,
        HTML("<small>Result will include a brief prose explanation of what was included/excluded.</small>")
    ],
    layout=Layout(width="260px", padding="10px", border="1px solid #eee")
)

chat_log = VBox(layout=Layout(border='1px solid #ddd', padding='8px', height='420px', overflow_y='auto', flex='1'))
user_input = Textarea(
    placeholder='Ask a question or request an analysis…',
    layout=Layout(width='100%', height='90px', resize='vertical')
)
send_btn = Button(description='Send', layout=Layout(width='120px', height='40px'))
clear_btn = Button(description='Clear', layout=Layout(width='120px', height='40px'))
controls = HBox([send_btn, clear_btn])

main = VBox([chat_log, user_input, controls], layout=Layout(flex='1'))
app = HBox([sidebar, main], layout=Layout(gap="12px"))

# --- Message helpers ---
def add_message(role, text):
    safe = html.escape(text).replace('\n', '<br>')
    color = '#eef6ff' if role == 'user' else '#f6f6f6'
    align = 'flex-end' if role == 'user' else 'flex-start'
    bubble = HTML(
        value=f"""
        <div style="display:flex; justify-content:{align};">
          <div style="
            max-width: 90%;
            margin: 6px 0;
            padding: 10px 12px;
            border-radius: 14px;
            background:{color};
            border:1px solid #e5e5e5;
            font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial;
            font-size: 14px; line-height: 1.45;">
            <b style="opacity:.6">{'You' if role=='user' else 'Bot'}</b><br>{safe}
          </div>
        </div>
        """,
        layout=Layout(width='100%')
    )
    chat_log.children = (*chat_log.children, bubble)

# --- Prompt template (format-aware) ---
def build_messages(user_question: str, chosen_format: str):
    """
    Template goals:
      1) Answer the user's question.
      2) Produce results in the chosen format (csv/json/markdown).
      3) Always include a short prose explanation of inclusion/exclusion criteria.
      4) Perform an internal reflective quality check; correct errors before answering.
         Do not reveal internal reasoning—only provide the final answer.
    """
    # Normalize label for instructions
    fmt_label = chosen_format.upper()
    # Simple, explicit directions for structure
    format_instructions = {
        "csv": textwrap.dedent("""\
            Output Format: CSV
            - Provide ONLY one CSV block for the results with a single header row.
            - Use commas as separators; escape commas/newlines/quotes correctly.
            - After the CSV block, include a short prose paragraph titled "Explanation".
        """),
        "json": textwrap.dedent("""\
            Output Format: JSON
            - Provide ONLY one valid JSON object or array for the results.
            - Ensure valid JSON (proper quotes, no trailing commas).
            - After the JSON, include a short prose paragraph titled "Explanation".
        """),
        "markdown": textwrap.dedent("""\
            Output Format: Markdown
            - Provide results as a well-formed Markdown table (with a header row).
            - After the table, include a short prose paragraph titled "Explanation".
        """),
    }[chosen_format]

    system_prompt = textwrap.dedent("""\
        You are a meticulous analyst. Be concise and accurate. When asked to output in a specific format,
        strictly follow that format. Always include a short prose explanation of inclusion/exclusion
        decisions. Perform an internal reflective quality check to avoid errors and correct them before
        producing the final answer. Do NOT reveal your internal reasoning; provide only the final answer.
    """)

    user_prompt = textwrap.dedent(f"""\
        User Question:
        {user_question}

        Requirements:
        - Answer the question.
        - Produce results in {fmt_label} as specified below.
        - Always include a prose "Explanation" describing why certain results were included or excluded.
        - Perform a reflective quality check; correct any mistakes silently before finalizing.

        {format_instructions}
    """)

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def call_azure_openai(messages):
    completion = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,  # deployment name
        messages=messages,
        temperature=0.2,                 # lower temp for more deterministic, schema-friendly output
    )
    return completion.choices[0].message.content

# --- Event handlers ---
def handle_send(_=None):
    text = user_input.value.strip()
    if not text:
        return
    send_btn.disabled = True
    try:
        # Show user message (with format selection summary)
        fmt = format_picker.value
        add_message('user', f"{text}\n\n[Requested format: {fmt.upper()}]")

        # Build messages with format-aware template
        msgs = build_messages(text, fmt)

        # Call model
        reply = call_azure_openai(msgs)

        # Show bot reply as-is (it contains the formatted result + Explanation)
        add_message('bot', reply)
        user_input.value = ""
        chat_log.layout.overflow_y = 'auto'
    finally:
        send_btn.disabled = False

def handle_clear(_):
    chat_log.children = ()

send_btn.on_click(handle_send)
clear_btn.on_click(handle_clear)

display(app)